<a href="https://colab.research.google.com/github/anujott-codes/IOSC-PIMA-INDIAN/blob/main/IOSC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#IOSC PROJECT
#*PIMA-INDIAN DIABETES CLASSIFICATION*